In [3]:
import os
import re
import subprocess

import mgitools.bsub as bsub
import mgitools.os_helpers as os_helpers

In [13]:
BAM_MAP_LOCATION = '/gscuser/estorrs/CPTAC3.catalog/MGI.BamMap.dat'
# BAM_MAP_LOCATION = '/gscuser/estorrs/gbm.bobo.bammap.dat'
EXECUTION_DIR = '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution'
LOG_DIR = '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/indexing'

## heirarchy and symlinking

In [15]:
f = open(BAM_MAP_LOCATION)
sample_data = {}


identifiers = [r'WXS', 'hg38', 'BAM', r'blood_normal|tumor', 'PDA']
# identifiers = ['blood_normal']


# filter based on identifiers and get sample name and location
for line in f:
    is_valid = True
    for identifier in identifiers:
        if not re.findall(identifier, line):
            is_valid = False
            break

    if is_valid:
        pieces = line.strip().split('\t')

        sample = pieces[1]
        technology = pieces[3]
        sample_type = pieces[4]
        fp = pieces[5]
        
        if sample_type == 'blood_normal':
            sample = sample + '.N'
        else:
            sample = sample + '.T'
        
        if sample in sample_data:
            sample_data[sample]
        
        if sample in sample_data:
            d = sample_data[sample]
            if d['technology'].lower() != 'wxs' and technology.lower() == 'wxs':
                sample_data[sample] = {'technology': technology, 'fp': fp}
        else:
            sample_data[sample] = {'technology': technology, 'fp': fp}

len(sample_data), list(sample_data.items())[:5]

(154,
 [('C3L-00017.N',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_import/data/f99797be-7884-4f95-9052-7637fe7de122/6aa9462a-bda4-4c7e-9049-dfd4e79fab18_gdc_realn.bam'}),
  ('C3L-00017.T',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_import/data/83ea0e10-82b1-4092-b8a4-770e179ef4cd/90daf6b6-b405-431c-bb0d-6f52df3e6ef9_gdc_realn.bam'}),
  ('C3L-00102.N',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_import/data/a4c9d5ea-87f2-4812-bf9e-2a21e0fbd265/b6b55db6-b2f3-4d98-b1d1-3958bf5ae28f_gdc_realn.bam'}),
  ('C3L-00102.T',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_import/data/e4e743cb-4e2d-4ef3-92e9-b6f538ded0c9/945dbda3-ac53-448a-abd8-96ebc5c55b48_gdc_realn.bam'}),
  ('C3L-00189.N',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_import/data/82b6d795-c9

In [16]:
sample_to_fps = {}
for sample, d in sample_data.items():
    s_id = sample[:-2]
    
    if s_id not in sample_to_fps:
        sample_to_fps[s_id] = {}
    
    if sample[-1] == 'N':
        sample_to_fps[s_id]['normal'] = d['fp']
    else:
        sample_to_fps[s_id]['tumor'] = d['fp']
len(sample_to_fps), list(sample_to_fps.items())[:5]

(77,
 [('C3L-00017',
   {'normal': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_import/data/f99797be-7884-4f95-9052-7637fe7de122/6aa9462a-bda4-4c7e-9049-dfd4e79fab18_gdc_realn.bam',
    'tumor': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_import/data/83ea0e10-82b1-4092-b8a4-770e179ef4cd/90daf6b6-b405-431c-bb0d-6f52df3e6ef9_gdc_realn.bam'}),
  ('C3L-00102',
   {'normal': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_import/data/a4c9d5ea-87f2-4812-bf9e-2a21e0fbd265/b6b55db6-b2f3-4d98-b1d1-3958bf5ae28f_gdc_realn.bam',
    'tumor': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_import/data/e4e743cb-4e2d-4ef3-92e9-b6f538ded0c9/945dbda3-ac53-448a-abd8-96ebc5c55b48_gdc_realn.bam'}),
  ('C3L-00189',
   {'normal': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_import/data/82b6d795-c960-42e3-a2f6-fa488f4f4d3e/7c72d1eb-35b2-4f3b-ac73-db99b0c1832c_gdc_realn.bam',
    'tumor': '/gscmnt/gc2521/dinglab/mwyczalk/somatic-wrapper-data/GDC_impo

In [17]:
for sample, d in sample_to_fps.items():
    # make directory for sample
    os.mkdir(os.path.join(EXECUTION_DIR, sample))
    
    # link files in directory
    for k, fp in d.items():
        if k == 'normal':
            tool_args = ('ln', '-s', fp, os.path.join(EXECUTION_DIR, sample, f'{sample}.N.bam'))
        else:
            tool_args = ('ln', '-s', fp, os.path.join(EXECUTION_DIR, sample, f'{sample}.T.bam'))
            
        subprocess.check_output(tool_args)

## indexing

In [18]:
regex = r'.bam$'
    
fps = list(os_helpers.listfiles(EXECUTION_DIR,
                regex=regex))

sample_fps_tups = [(fp.split('/')[-1].replace('.bam', ''), fp) for fp in fps]

len(sample_fps_tups), sample_fps_tups[:5]

(154,
 [('C3N-01997.N',
   '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution/C3N-01997/C3N-01997.N.bam'),
  ('C3N-01997.T',
   '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution/C3N-01997/C3N-01997.T.bam'),
  ('C3N-00514.T',
   '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution/C3N-00514/C3N-00514.T.bam'),
  ('C3N-00514.N',
   '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution/C3N-00514/C3N-00514.N.bam'),
  ('C3N-01169.N',
   '/gscmnt/gc2508/dinglab/estorrs/cptac3/germline_calling_execution/pdac/hg38/execution/C3N-01169/C3N-01169.N.bam')])

In [19]:
image = 'estorrs/slate:0.0.2'

# os.mkdir(LOG_DIR)

commands, log_files = [], []
for s_id, fp in sample_fps_tups:

    command = f'samtools index {fp}'
    log_fp = os.path.join(LOG_DIR, s_id + '.log')

    commands.append(command)
    log_files.append(log_fp)
        
bsub.generate_bsub_bash_script(commands, image, os.path.join(LOG_DIR, 'commands.sh'),
                              min_memory=4, max_memory=5, log_files=log_files)

## writing executables